In [1]:
import os
import sys

sys.path.append("../../common/src")
sys.path.append("../src")

In [2]:
from common.models import Agent
from common.models.agent import AgentType

In [3]:
agent_type = AgentType.LANGCHAIN_CONVERSATIONAL

In [4]:
agent_class = AgentType.langchain_class_from_agent_type(agent_type)

In [5]:
agent_class

langchain.agents.conversational.base.ConversationalAgent